# 1. Basic State

In [45]:
from typing import TypedDict, List
from langgraph.graph import StateGraph, START, END

class SimpleState(TypedDict):
    count: int
    message: List[str]
    
def increment(state: SimpleState) -> SimpleState:
    new_message = state["message"] + ["count" + str(state["count"])]
    return {
        "count": state["count"] + 1,
        "message": new_message
    }

    
def should_continue(state):
    if(state["count"] < 5):
        return "continue"
    else:
        return "stop"

graph = StateGraph(SimpleState)
graph.add_node("increment", increment)
graph.add_edge(START, "increment")
graph.add_conditional_edges(
    "increment",
    should_continue,
    {
        "continue": "increment",
        "stop": END
    }
)

app = graph.compile()

state = {
    "count": 0,
    "message": []
}

app.invoke(state)


{'count': 5, 'message': ['count0', 'count1', 'count2', 'count3', 'count4']}

# Declarative Annotated State Transformation

In [52]:
from typing import TypedDict, List, Annotated
from langgraph.graph import StateGraph, START, END
import operator

class SimpleState(TypedDict):
    count: int
    sum: Annotated[int, operator.add]
    message: Annotated[List[str], operator.concat]
    
def increment(state: SimpleState) -> SimpleState:
    # new_message = state["message"] + ["count" + str(state["count"])]
    new_value = state["count"] + 1
    return {
        "count": new_value,
        "sum": state["count"],
        "message": [new_value]
    }

    
def should_continue(state):
    if(state["count"] < 5):
        return "continue"
    else:
        return "stop"

graph = StateGraph(SimpleState)
graph.add_node("increment", increment)
graph.add_edge(START, "increment")
graph.add_conditional_edges(
    "increment",
    should_continue,
    {
        "continue": "increment",
        "stop": END
    }
)

app = graph.compile()

state = {
    "count": 0,
    "sum": 0,
    "message": []
}

app.invoke(state)


{'count': 5, 'sum': 10, 'message': [1, 2, 3, 4, 5]}